In [1]:
import pandas as pd
import numpy as np
import math
import statsmodels.api as sm
# pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 200

In [2]:
data_13D = pd.read_csv(r'C:\Users\aleor\OneDrive\Documents\Berkeley\Econ_191\191_Data\13D_with_years.csv')

In [3]:
with_pscore = pd.read_stata(r"C:\Users\aleor\OneDrive\Documents\Berkeley\Econ_191\191_Data\By_Permno_Data\with_two_pscores.dta", convert_dates=False, convert_categoricals=False, convert_missing=False)

In [4]:
with_pscore['sic_2'] = with_pscore['sic']//100

In [5]:
with_pscore.shape

(189678, 66)

In [6]:
# find duplicates
with_pscore[with_pscore.duplicated(['permno', 'year']) == True].shape

(0, 66)

In [7]:
# matching starts here
indices1 =  with_pscore.index[with_pscore.loc[:, 'event_year_treatment'] == 1]
indices2 =  with_pscore.index[with_pscore.loc[:, 'event_year_treatment'] == 0]
list1 = [[with_pscore.iloc[i]['pscore_2'], with_pscore.iloc[i]['sic_2'], with_pscore.iloc[i]['year'], i] for i in indices1]
list2 = [[with_pscore.iloc[i]['pscore_2'], with_pscore.iloc[i]['sic_2'], with_pscore.iloc[i]['year'],i] for i in indices2]

In [8]:
len(list1)

2744

In [9]:
list1

[[0.028329998, 49, 1999, 9],
 [0.02875735, 49, 2014, 24],
 [nan, 38, 1997, 102],
 [0.032190565, 30, 2005, 149],
 [0.02989084, 30, 2008, 152],
 [0.029452609, 30, 2010, 154],
 [0.029680174, 35, 2000, 286],
 [0.02744548, 20, 1999, 324],
 [0.029827798, 80, 2005, 378],
 [nan, 50, 1997, 439],
 [0.009245626, 35, 2008, 530],
 [nan, 48, 1995, 830],
 [0.027953662, 27, 1998, 857],
 [0.028604323, 28, 1998, 967],
 [nan, 60, 2000, 1045],
 [0.023510272, 28, 1998, 1322],
 [nan, 35, 2010, 1462],
 [0.02625098, 35, 2014, 1466],
 [0.0116596725, 20, 2010, 1504],
 [0.028748037, 80, 2004, 1736],
 [0.0290888, 80, 2005, 1737],
 [0.029497195, 36, 2012, 1803],
 [0.029662028, 36, 2013, 1804],
 [nan, 60, 2007, 1835],
 [0.027510516, 36, 2006, 1852],
 [0.027698722, 36, 2010, 1856],
 [nan, 42, 1995, 1887],
 [0.026842404, 36, 2007, 2037],
 [0.027664356, 32, 2000, 2088],
 [0.029895527, 56, 1998, 2188],
 [0.028571527, 28, 2012, 2228],
 [nan, 38, 1996, 2266],
 [0.030746015, 38, 1998, 2268],
 [0.027962426, 38, 2005, 2275]

In [10]:
list1_no_nans = [i for i in list1 if not np.isnan(i[0])]
list2_no_nans = [i for i in list2 if not np.isnan(i[0])]

In [11]:
len(list1_no_nans)

1738

In [12]:
# function returns index of firm-year in control that minimizes propensity score difference 
def nearestDistance(set_2, point):
    minDistList = map(lambda x: np.absolute(x[0] - point), set_2)
    zipped_minDistList = zip(minDistList, set_2)
    minDist2 = min(zipped_minDistList)
    return minDist2[1][1]

In [14]:
# run loop for list1 - returns list with index of matched control first, then index of treatment
d = []
for p in list1_no_nans:
    list_indicies_to_pass_in = with_pscore[(with_pscore['year'] == p[2]) & (with_pscore['sic_2'] == p[1]) & (with_pscore['ever_target'] == 0)].index.values
    print(list_indicies_to_pass_in)
    if not list_indicies_to_pass_in.any():
        d.append('no match')
    else:
        formatted_list_to_pass_in = [[with_pscore.iloc[i]['pscore_2'], i] for i in list_indicies_to_pass_in]
        d.append([nearestDistance(formatted_list_to_pass_in, p[0]), p[3]])

[   893   3079   6968   7777   8394   8654  10090  12137  15549  15591
  18293  23422  24305  26144  26324  26865  28513  28579  28850  28961
  29106  29200  29217  29630  29671  29719  29783  29989  30089  30119
  30173  30225  30253  30295  30324  30540  30566  30613  30743  30848
  30858  30892  30909  30979  31054  31066  31161  31200  31245  31285
  31314  31336  31365  31426  31446  31475  31492  31541  31628  31714
  31980  32009  32159  32252  32429  32833  33103  33132  33287  34121
  34337  34424  35722  35828  35969  36863  37341  37352  38385  38881
  39562  40461  40625  40706  41963  43028  43945  44259  47340  48534
  49161  49699  49761  50853  51624  51775  54051  55143  55209  55651
  56844  56889  58216  58461  58495  58950  59378  60035  60275  60462
  60832  60963  61009  61055  61130  61240  61551  64024  67604  67621
  67836  71773  71882  73902  73950  77826  77901  81113  82674  84669
  86294  89239  91888  95161  95208  95344  98242 105566 107739 108391
 10878

[   590    769   1193   1760   1865   2138   2166   2731   3196   3320
   3591   4133   4253   4409   4472   5061   5187   5681   5897   6042
   6421   6804   7085   7529   8193   8488   9216   9540   9632   9735
   9950  10273  10326  10480  11413  11496  11990  12554  13325  13392
  13669  13823  17612  18985  23650  23671  24123  26639  26679  26821
  26929  27112  27321  27462  28076  28215  28398  28695  29250  29325
  29433  29641  29841  30024  31034  31179  31684  31797  32043  33159
  33592  34248  34613  34651  35274  35330  35433  35444  36364  36404
  36615  36901  37383  38165  39338  39414  39972  40057  40272  40333
  41379  41461  41503  41523  41953  42446  42579  42898  43183  43507
  43555  43673  44469  44528  44643  44686  45178  45224  46217  46919
  48299  49081  49190  52508  52517  53170  54003  55836  56801  57198
  59462  59897  60762  61431  62045  62631  63670  64233  65264  65560
  65574  65799  65870  66874  66899  68364  69003  69168  69627  69701
  7099

[   238   2011   3405   6869  11519  12473  12778  14045  14287  14425
  14429  14761  14769  20861  23211  23486  24333  28439  29311  29524
  32411  32927  35123  36579  37594  38564  39061  39889  42003  42306
  44550  44848  45296  46128  46713  47201  47327  48730  49260  49287
  49445  50045  51191  51533  51590  51680  52578  53270  54568  54787
  55593  56138  57413  57790  58334  59203  60245  62186  64799  68128
  68294  68884  69231  74532  74805  75753  76963  79203  79893  80362
  86246  89000  89492  91936  92262  92869  93126  93270  94167  96096
  96597  97431  98488 100388 100424 100744 100796 101542 101679 102553
 102669 104760 107014 107892 109166 111024 111089 112501 112928 113207
 113655 114017 117240 117420 118679 119591 119826 121635 122293 123213
 123598 123874 124115 125852 125942 127572 127643 130146 133452 134008
 134367 136447 136541 136703 136797 136842 137050 137285 137421 137663
 138170 139068 139807 142428 142455 142852 142979 143041 144369 144611
 14513

[  1118   3179   3938   6710   7038   8259   8469   8863   9394   9518
  10594  11037  11545  11731  12944  12972  13587  24443  26575  29411
  29941  31843  31936  32525  33544  35059  35634  35884  36206  36518
  36769  39867  40044  41154  42658  43161  44071  44506  45112  46079
  46203  46469  46680  47378  48363  49050  49534  50261  50493  51044
  51098  51977  52101  54345  55365  56822  58047  58676  59276  59404
  60166  61891  62338  63166  65080  67978  68742  70223  72770  73979
  75430  77123  77699  80308  80713  81515  81557  81738  82845  89846
  89939  90407  90496  90712  91439  92739  93144  93802  94193  95747
  98276  98552  98580  98995  99065 100445 102504 104358 107298 108064
 108881 109438 116028 119758 120358 120555 122452 123045 123140 123931
 125170 126237 126318 126747 127280 128484 128599 128779 129001 129216
 131391 131570 131630 131901 131908 132165 135679 136866 137329 137714
 137854 138500 139539 139640 140618 140994 141221 141736 142785 143018
 14333

[  1120   3181   3940   6712   8261   9396   9520  10596  11039  11547
  11733  13589  24445  29413  29943  31845  31938  32527  33546  35061
  35636  35886  36208  36771  40046  41156  42660  43163  44073  45114
  46081  46205  46471  46682  47380  48365  49052  49536  50263  51046
  51100  51979  52103  55367  56824  58049  58678  59406  60168  61893
  62340  63168  65082  67980  68744  70225  72772  75432  77701  80310
  80715  81517  81559  81740  82847  89848  89941  90409  90498  90714
  91441  92741  93146  95749  98278  98554  98582  98997  99067 100447
 102506 104360 107300 108066 108883 109440 116030 120557 122454 123142
 126320 126749 128486 128601 128781 131393 131572 131903 131910 132167
 136868 137716 139541 139642 140620 140996 141223 142787 143333 143533
 143762 146791 149964 151149 151609 151663 151685 152962 153268 156071
 156211 156303 156323 156853 157098 157185 157617 158804 159663 159728
 159821 160276 160421 160634 160707 160817 160850 161204 161328 162156
 16218

...

[   694    726   1706   2362   4340   5158   5576   8379   8815  10032
  11178  12268  12315  13962  14007  14160  14168  14191  14200  14239
  14248  14324  14509  14544  14575  14641  14693  14719  14747  14917
  14969  15134  15251  15260  15276  15292  15462  15500  15658  15708
  15731  15767  15782  15802  15972  15976  16228  16310  16324  16464
  16496  16504  16520  16582  16598  16632  16644  16741  16753  16825
  16846  16941  17013  17034  17044  17138  17170  17204  17310  17328
  17348  17457  17482  17535  17550  17565  17641  17671  17683  17695
  17922  17941  18185  18264  18269  18325  18336  18357  18449  18499
  18505  18611  18701  18707  18718  18768  18781  18936  18967  18970
  19012  19135  19152  19183  19206  19223  19342  19359  19388  19394
  19404  19428  19445  19598  19700  19742  19748  19780  19828  19838
  19928  20002  20039  20086  20111  28112  32872  35950  36714  41434
  42117  44329  45072  46514  46543  46904  50240  61923  68102  71718
  7489

[   238   2011   3405   6869  11519  12473  12778  14045  14287  14425
  14429  14761  14769  20861  23211  23486  24333  28439  29311  29524
  32411  32927  35123  36579  37594  38564  39061  39889  42003  42306
  44550  44848  45296  46128  46713  47201  47327  48730  49260  49287
  49445  50045  51191  51533  51590  51680  52578  53270  54568  54787
  55593  56138  57413  57790  58334  59203  60245  62186  64799  68128
  68294  68884  69231  74532  74805  75753  76963  79203  79893  80362
  86246  89000  89492  91936  92262  92869  93126  93270  94167  96096
  96597  97431  98488 100388 100424 100744 100796 101542 101679 102553
 102669 104760 107014 107892 109166 111024 111089 112501 112928 113207
 113655 114017 117240 117420 118679 119591 119826 121635 122293 123213
 123598 123874 124115 125852 125942 127572 127643 130146 133452 134008
 134367 136447 136541 136703 136797 136842 137050 137285 137421 137663
 138170 139068 139807 142428 142455 142852 142979 143041 144369 144611
 14513

[   695    727   1707   2363   4341   5159   5577   8380   8816  10033
  11179  12316  13963  14008  14161  14169  14192  14201  14240  14249
  14325  14510  14545  14642  14665  14694  14720  14748  14918  14970
  15135  15252  15261  15277  15293  15463  15501  15659  15709  15732
  15768  15783  15803  15973  15977  16229  16311  16325  16465  16497
  16505  16521  16583  16633  16645  16742  16754  16826  16847  16942
  17014  17035  17045  17139  17171  17205  17311  17329  17458  17483
  17536  17551  17566  17642  17672  17684  17696  17923  17942  18186
  18265  18270  18326  18337  18358  18450  18500  18506  18612  18702
  18708  18719  18769  18782  18937  18968  18971  19013  19153  19184
  19207  19224  19343  19360  19389  19395  19429  19446  19599  19701
  19743  19749  19781  19829  19839  19929  20003  20040  20087  20112
  20164  20197  20359  20471  20475  20525  20535  20545  20555  20567
  20577  20592  20661  20699  20722  20748  20787  20811  20813  20818
  2087

[  1347   8409  10105  14624  15021  15564  15606  15847  16140  16398
  17191  17874  18027  18618  19007  19046  19119  19218  19463  19923
  20124  20243  20550  20772  21092  21160  21316  21648  21653  21817
  21964  22360  23437  26339  26880  28528  28865  29121  29232  29686
  30134  30188  30240  30268  30310  30339  30555  30581  30758  30873
  30994  31081  31215  31300  31351  31461  31556  31995  32024  32174
  32267  33118  33147  33302  34136  34352  34439  37367  39577  40640
  41978  43960  48549  49176  49776  50868  51639  51790  55158  55666
  56859  58510  60290  61024  61145  61566  64039  67636  81127  82689
  84684  89254  98257 107754 108799 113361 116674 118889 118917 119280
 119403 122205 125483 130653 133362 133603 137256 140854 142627 143373
 147431 147790 148480 155174 156929 165191 166940 168948 174303 175162
 177707 179195 179890 180287 182509 183220 183741 184515 184634 186572
 186702 186831 186898 187469 188299 188345 189437]
[  1346   8408  10104  146

[   242   2015   3409   6873  11523  12782  14049  14291  14433  14685
  14765  14773  15029  15244  15370  15617  15639  15968  16025  16189
  16236  16333  16668  16840  16889  17018  17120  17707  17964  19380
  19476  19737  20011  20490  20757  20865  21100  21109  21124  21378
  22014  22058  23215  23490  24337  28443  29528  32415  32931  36583
  37598  38568  39065  39893  42007  42310  44554  44852  45300  46717
  47205  48734  49291  49449  50049  51195  51537  51594  51684  53274
  54572  54791  55597  56142  57417  57794  58338  60249  62190  69235
  74536  74809  75757  76967  79207  79897  89004  89496  91940  92266
  93130  93274  94171  96100  96601  98191  98492 100428 100800 101546
 101683 102557 102673 104764 107018 107896 111028 111093 112505 112932
 113211 113659 114021 117244 117421 118683 119830 121639 122297 123602
 123878 124697 125856 125946 127576 127647 130150 133456 134012 136451
 136545 136846 137054 137289 137425 137667 139072 139811 142432 142459
 14285

In [15]:
d

[[28513, 9],
 [119403, 24],
 [70355, 149],
 [33750, 152],
 [182613, 154],
 [132154, 286],
 [91425, 324],
 [107061, 378],
 [53617, 530],
 [58292, 857],
 [89874, 967],
 [77047, 1322],
 [99238, 1466],
 [53452, 1504],
 [157503, 1736],
 [157504, 1737],
 [187282, 1803],
 [152679, 1804],
 [96092, 1852],
 [186146, 1856],
 [154233, 2037],
 [12906, 2088],
 [1653, 2188],
 [4147, 2228],
 [44627, 2268],
 [108881, 2275],
 [34592, 2334],
 [24123, 2411],
 [144681, 2423],
 [5200, 2424],
 [157185, 2462],
 [188833, 2526],
 [174271, 2548],
 [61918, 2703],
 [42810, 3091],
 [148328, 3216],
 [51104, 3435],
 [141320, 3543],
 [163931, 3630],
 [89749, 3631],
 [151699, 3632],
 [115604, 3732],
 [90415, 3970],
 [36981, 4184],
 [165270, 4302],
 [165646, 4304],
 [157739, 4736],
 [189276, 4737],
 [127569, 4804],
 [162383, 4805],
 [23199, 5223],
 [155921, 5300],
 [97030, 5384],
 [159375, 5426],
 [96394, 5428],
 [68909, 5993],
 [148324, 6078],
 [89686, 6269],
 [8931, 6477],
 [84071, 6748],
 [153267, 6895],
 [148061, 74

In [16]:
list_with_indices_permno_codes = [[with_pscore.iloc[i[0]]['permno'], i[0], with_pscore.iloc[i[1]]['permno'], i[1]] for i in d]

In [17]:
list_with_indices_permno_codes

[[20750, 28513, 10001, 9],
 [81084, 119403, 10001, 24],
 [75150, 70355, 10025, 149],
 [27430, 33750, 10025, 152],
 [91664, 182613, 10025, 154],
 [83685, 132154, 10039, 286],
 [77446, 91425, 10044, 324],
 [79580, 107061, 10051, 378],
 [56573, 53617, 10078, 530],
 [61743, 58292, 10127, 857],
 [77274, 89874, 10143, 967],
 [75860, 77047, 10200, 1322],
 [78664, 99238, 10220, 1466],
 [56274, 53452, 10225, 1504],
 [87637, 157503, 10257, 1736],
 [87637, 157504, 10257, 1737],
 [92758, 187282, 10259, 1803],
 [86929, 152679, 10259, 1804],
 [77918, 96092, 10272, 1852],
 [92460, 186146, 10272, 1856],
 [87139, 154233, 10302, 2037],
 [12107, 12906, 10307, 2088],
 [10247, 1653, 10325, 2188],
 [10661, 4147, 10333, 2228],
 [45102, 44627, 10342, 2268],
 [79776, 108881, 10342, 2275],
 [28223, 34592, 10353, 2334],
 [15991, 24123, 10363, 2411],
 [85744, 144681, 10363, 2423],
 [10860, 5200, 10363, 2424],
 [87584, 157185, 10371, 2462],
 [93191, 188833, 10382, 2526],
 [90333, 174271, 10383, 2548],
 [65270, 619

In [18]:
with_pscore['event_year_control'] = 0
with_pscore['permno_for_matched_control'] = 0
with_pscore['index_for_matched_control'] = 0

In [19]:
for i in list_with_indices_permno_codes:
    print(i)
    with_pscore.ix[i[1], 'event_year_control'] = 1
    with_pscore.ix[i[1], 'permno_for_matched_control'] = i[2]
    with_pscore.ix[i[1], 'index_for_matched_control'] = i[3]

[20750, 28513, 10001, 9]
[81084, 119403, 10001, 24]
[75150, 70355, 10025, 149]
[27430, 33750, 10025, 152]
[91664, 182613, 10025, 154]
[83685, 132154, 10039, 286]


C:\Users\aleor\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\aleor\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
C:\Users\aleor\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-depreca

[77446, 91425, 10044, 324]
[79580, 107061, 10051, 378]
[56573, 53617, 10078, 530]
[61743, 58292, 10127, 857]
[77274, 89874, 10143, 967]
[75860, 77047, 10200, 1322]
[78664, 99238, 10220, 1466]
[56274, 53452, 10225, 1504]
[87637, 157503, 10257, 1736]
[87637, 157504, 10257, 1737]
[92758, 187282, 10259, 1803]
[86929, 152679, 10259, 1804]
[77918, 96092, 10272, 1852]
[92460, 186146, 10272, 1856]
[87139, 154233, 10302, 2037]
[12107, 12906, 10307, 2088]
[10247, 1653, 10325, 2188]
[10661, 4147, 10333, 2228]
[45102, 44627, 10342, 2268]
[79776, 108881, 10342, 2275]
[28223, 34592, 10353, 2334]
[15991, 24123, 10363, 2411]
[85744, 144681, 10363, 2423]
[10860, 5200, 10363, 2424]
[87584, 157185, 10371, 2462]
[93191, 188833, 10382, 2526]
[90333, 174271, 10383, 2548]
[65270, 61918, 10404, 2703]
[42439, 42810, 10488, 3091]
[86209, 148328, 10514, 3216]
[53110, 51104, 10550, 3435]
[85298, 141320, 10568, 3543]
[89004, 163931, 10586, 3630]
[77262, 89749, 10586, 3631]
[86791, 151699, 10586, 3632]
[80575, 1156

[61241, 57779, 37196, 39597]
[76195, 79823, 37381, 39748]
[10866, 5246, 37381, 39750]
[92614, 186830, 38156, 40233]
[76961, 86862, 38172, 40263]
[81208, 120405, 38172, 40264]
[60468, 57084, 38295, 40292]
[10180, 1209, 38420, 40359]
[75566, 74459, 38578, 40419]
[80621, 115904, 38682, 40520]
[80621, 115906, 38682, 40522]
[49744, 48423, 38850, 40682]
[82449, 125153, 38973, 40761]
[52090, 50169, 38973, 40771]
[88596, 161430, 39029, 40802]
[80924, 117983, 39087, 40844]
[80012, 111019, 39220, 40870]
[69446, 65838, 39256, 40923]
[12049, 12430, 39256, 40925]
[73219, 68783, 39693, 41186]
[82762, 127570, 39757, 41253]
[46893, 46103, 39773, 41303]
[80581, 115676, 40061, 41363]
[84621, 138385, 40440, 41571]
[79944, 110538, 40468, 41598]
[27617, 33894, 40468, 41599]
[22374, 29573, 40694, 41758]
[85213, 140667, 40707, 41791]
[81713, 123014, 41064, 41882]
[31668, 36549, 41241, 42070]
[81003, 118591, 41320, 42131]
[84419, 137410, 41371, 42192]
[76023, 78236, 41419, 42227]
[11183, 7036, 41574, 42349]
[

[85177, 140386, 75320, 72100]
[77448, 91467, 75432, 73289]
[80343, 113773, 75434, 73328]
[60580, 57174, 75460, 73610]
[87251, 154904, 75460, 73616]
[81275, 120865, 75470, 73757]
[81540, 121660, 75489, 73929]
[30681, 35948, 75510, 74093]
[89207, 165279, 75526, 74196]
[85351, 141735, 75545, 74269]
[89540, 168131, 75573, 74500]
[77967, 96292, 75578, 74550]
[81028, 118769, 75596, 74727]
[90901, 178496, 75596, 74732]
[48072, 47205, 75596, 74738]
[91829, 183252, 75646, 75299]
[62498, 59249, 75649, 75336]
[82250, 124624, 75684, 75580]
[27430, 33748, 75729, 75819]
[20336, 28111, 75853, 76935]
[77979, 96413, 75861, 77078]
[24248, 31342, 75888, 77316]
[86991, 153085, 75993, 78040]
[81137, 119902, 75993, 78042]
[78876, 100449, 76020, 78202]
[89031, 164188, 76052, 78391]
[77454, 91523, 76085, 78656]
[77454, 91526, 76085, 78659]
[82924, 128295, 76085, 78663]
[23182, 30302, 76091, 78758]
[81225, 120563, 76095, 78797]
[90942, 178764, 76100, 78870]
[79797, 109155, 76111, 78959]
[13119, 16383, 76111, 7

[23835, 30857, 79999, 110856]
[92197, 184899, 80001, 110891]
[13106, 16311, 80001, 110895]
[78216, 98526, 80023, 111124]
[79686, 108094, 80034, 111231]
[80679, 116316, 80034, 111234]
[90901, 178501, 80049, 111328]
[19271, 27396, 80054, 111381]
[79172, 103315, 80076, 111530]
[88590, 161367, 80114, 111926]
[88590, 161368, 80114, 111927]
[79149, 103091, 80126, 111994]
[76745, 85021, 80154, 112172]
[72128, 68126, 80165, 112246]
[79588, 107162, 80167, 112274]
[75881, 77194, 80181, 112442]
[65700, 62511, 80204, 112675]
[65700, 62512, 80204, 112676]
[65700, 62517, 80204, 112681]
[84343, 136798, 80236, 112904]
[88729, 162025, 80236, 112906]
[79121, 102779, 80252, 113047]
[86715, 151168, 80257, 113107]
[76846, 85980, 80258, 113119]
[77443, 91413, 80258, 113120]
[89932, 171432, 80297, 113382]
[89297, 166030, 80297, 113383]
[89916, 171297, 80297, 113386]
[12190, 13396, 80306, 113448]
[81885, 123716, 80307, 113469]
[11315, 7677, 80320, 113570]
[54594, 52345, 80320, 113579]
[89029, 164156, 80341, 1

[13103, 16289, 84041, 134633]
[90031, 172011, 84052, 134713]
[87816, 158167, 84052, 134714]
[84320, 136613, 84057, 134760]
[12097, 12845, 84062, 134813]
[34287, 38023, 84069, 134852]
[16791, 25284, 84103, 135038]
[87415, 156074, 84161, 135421]
[87601, 157265, 84172, 135531]
[88742, 162103, 84176, 135579]
[62885, 59504, 84198, 135734]
[51984, 50081, 84203, 135763]
[85067, 140009, 84210, 135844]
[26084, 32869, 84255, 136138]
[89387, 166790, 84255, 136142]
[82686, 127005, 84296, 136403]
[85306, 141367, 84296, 136404]
[53453, 51360, 84312, 136518]
[75828, 76595, 84372, 136977]
[83761, 132618, 84375, 137010]
[83779, 132731, 84385, 137086]
[92221, 185040, 84385, 137089]
[86197, 148277, 84387, 137117]
[76359, 81414, 84402, 137271]
[26084, 32869, 84415, 137378]
[93237, 188932, 84521, 137681]
[88550, 161151, 84531, 137737]
[92302, 185461, 84531, 137747]
[79251, 104211, 84534, 137754]
[87385, 155896, 84545, 137812]
[92432, 185980, 84546, 137821]
[79528, 106622, 84559, 137901]
[13777, 18179, 8455

[87168, 154438, 87589, 157220]
[88478, 160652, 87595, 157245]
[79121, 102779, 87611, 157352]
[79490, 106281, 87619, 157413]
[89956, 171615, 87632, 157455]
[89956, 171616, 87632, 157456]
[81527, 121535, 87633, 157475]
[93123, 188606, 87771, 157932]
[78868, 100377, 87777, 157953]
[83889, 133476, 87793, 158028]
[88360, 159871, 87795, 158034]
[81282, 120928, 87806, 158090]
[87762, 157894, 87806, 158091]
[65270, 61912, 87814, 158140]
[82212, 124378, 87814, 158144]
[87508, 156793, 87814, 158145]
[90708, 177117, 87814, 158147]
[75603, 74804, 87825, 158199]
[84157, 135374, 87827, 158220]
[88485, 160687, 88161, 158644]
[12312, 14007, 88166, 158680]
[51693, 49833, 88172, 158714]
[83762, 132630, 88177, 158774]
[92046, 184238, 88208, 159043]
[86560, 150709, 88246, 159170]
[79580, 107063, 88255, 159216]
[33209, 37478, 88255, 159219]
[78213, 98489, 88260, 159265]
[12217, 13588, 88263, 159286]
[87061, 153600, 88264, 159297]
[37217, 39627, 88281, 159420]
[90012, 171912, 88283, 159455]
[10138, 927, 883

[12449, 14544, 91619, 182358]
[10065, 474, 91627, 182443]
[10065, 475, 91627, 182444]
[92221, 185039, 91666, 182641]
[82212, 124386, 91666, 182642]
[88159, 158627, 91675, 182723]
[12052, 12467, 91733, 182933]
[12397, 14395, 91734, 182945]
[90276, 173791, 91754, 183061]
[10853, 5156, 91754, 183062]
[77186, 89130, 91826, 183239]
[91662, 182605, 91831, 183261]
[88807, 162385, 91836, 183297]
[20670, 28442, 91836, 183300]
[92448, 186057, 91836, 183301]
[90956, 178891, 91889, 183514]
[90805, 177859, 91892, 183526]
[85040, 139842, 91900, 183567]
[78910, 100745, 91901, 183581]
[38149, 40207, 91909, 183645]
[81696, 122861, 91937, 183752]
[92558, 186610, 91957, 183814]
[85675, 144118, 92101, 184432]
[82734, 127371, 92102, 184442]
[88873, 162895, 92185, 184839]
[10853, 5158, 92185, 184842]
[11003, 6096, 92215, 185008]
[93012, 188159, 92227, 185092]
[85298, 141322, 92297, 185429]
[27334, 33680, 92326, 185580]
[83875, 133358, 92389, 185807]
[10138, 927, 92402, 185877]
[11003, 6094, 92421, 185905]
[

In [20]:
with_pscore

,v1,gvkey,permno,datadate,year,indfmt,consol,popsrc,datafmt,cusip,...,v60,event_year_treatment,pscore_original,pscore_1,pscore_2,ever_target,sic_2,event_year_control,permno_for_matched_control,index_for_matched_control
0,0,12994,10001,19900630,1990,INDL,C,D,STD,367204104,...,NaN,0.0,NaN,NaN,NaN,1.0,49,0,0,0
1,1,12994,10001,19910630,1991,INDL,C,D,STD,367204104,...,NaN,0.0,NaN,NaN,NaN,1.0,49,0,0,0
2,2,12994,10001,19920630,1992,INDL,C,D,STD,367204104,...,NaN,0.0,NaN,NaN,NaN,1.0,49,0,0,0
3,3,12994,10001,19930630,1993,INDL,C,D,STD,367204104,...,NaN,0.0,NaN,NaN,NaN,1.0,49,0,0,0
4,4,12994,10001,19940630,1994,INDL,C,D,STD,367204104,...,NaN,0.0,NaN,NaN,NaN,1.0,49,0,0,0
5,5,12994,10001,19950630,1995,INDL,C,D,STD,367204104,...,NaN,0.0,NaN,NaN,NaN,1.0,49,0,0,0
6,6,12994,10001,19960630,1996,INDL,C,D,STD,367204104,...,NaN,0.0,NaN,NaN,NaN,1.0,49,0,0,0
7,7,12994,10001,19970630,1997,INDL,C,D,STD,367204104,...,NaN,0.0,NaN,NaN,NaN,1.0,49,0,0,0
8,8,12994,10001,19980630,1998,INDL,C,D,STD,367204104,...,NaN,0.0,0.030664,0.028835,0.028407,1.0,49,0,0,0
9,9,12994,10001,19990630,1999,INDL,C,D,STD,367204104,...,12994.0,1.0,0.030610,0.028769,0.028330,1.0,49,0,0,0


In [21]:
# make list of unique permnos
permnos = with_pscore['permno']
permno = []
for i in permnos:
    if i not in permno:
        permno.append(i)

In [22]:
# define relevant t5 (ie: within five years of event/pseudo-event)
with_pscore['relevant_t5'] = 0
for i in permno:
    list_indices = with_pscore[(with_pscore['permno'] == i) & ((with_pscore['event_year_treatment'] == 1)|(with_pscore['event_year_control'] == 1))].index.values
    if any(list_indices):
        for r in list_indices:
            for index in range(r-5, r+6):
                if with_pscore.iloc[index]['permno'] == i:
                    with_pscore.ix[index, 'relevant_t5'] = 1

C:\Users\aleor\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [34]:
# find post years for interval = 5 (ie: post event/pseudo-event and within 5 years of either)
with_pscore['post_t5'] = 0
for i in permno:
    list_indices = with_pscore[(with_pscore['permno'] == i) & (with_pscore['relevant_t5'] == 1) & ((with_pscore['event_year_treatment'] == 1)|(with_pscore['event_year_control'] == 1))].index.values
    if any(list_indices):
        min_index = min(list_indices)
        for index in range(min_index, min_index+25):
            if with_pscore.iloc[index]['permno'] == i:
                with_pscore.ix[index, 'post_t5'] = 1
            

C:\Users\aleor\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [35]:
# define relevant t3 (ie: within three years of event/pseudo-event)
with_pscore['relevant_t3'] = 0
for i in permno:
    list_indices = with_pscore[(with_pscore['permno'] == i) & ((with_pscore['event_year_treatment'] == 1)|(with_pscore['event_year_control'] == 1))].index.values
    if any(list_indices):
        for r in list_indices:
            for index in range(r-3, r+4):
                if with_pscore.iloc[index]['permno'] == i:
                    with_pscore.ix[index, 'relevant_t3'] = 1

C:\Users\aleor\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [36]:
# find post years for interval = 3 (ie: post event/pseudo-event and within 3 years of either)
with_pscore['post_t3'] = 0
for i in permno:
    list_indices = with_pscore[(with_pscore['permno'] == i) & (with_pscore['relevant_t3'] == 1) & ((with_pscore['event_year_treatment'] == 1)|(with_pscore['event_year_control'] == 1))].index.values
    if any(list_indices):
        min_index = min(list_indices)
        for index in range(min_index, min_index+25):
            if with_pscore.iloc[index]['permno'] == i:
                with_pscore.ix[index, 'post_t3'] = 1
            

C:\Users\aleor\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [37]:
# define relevant t7 (ie: within seven years of event/pseudo-event)
with_pscore['relevant_t7'] = 0
for i in permno:
    list_indices = with_pscore[(with_pscore['permno'] == i) & ((with_pscore['event_year_treatment'] == 1)|(with_pscore['event_year_control'] == 1))].index.values
    if any(list_indices):
        for r in list_indices:
            for index in range(r-7, r+8):
                if with_pscore.iloc[index]['permno'] == i:
                    with_pscore.ix[index, 'relevant_t7'] = 1

C:\Users\aleor\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [43]:
with_pscore

,v1,gvkey,permno,datadate,year,indfmt,consol,popsrc,datafmt,cusip,...,ever_target,sic_2,event_year_control,permno_for_matched_control,index_for_matched_control,relevant_t5,relevant_t3,relevant_t7,post_t5,post_t3
0,0,12994,10001,19900630,1990,INDL,C,D,STD,367204104,...,1.0,49,0,0,0,0,0,0,0,0
1,1,12994,10001,19910630,1991,INDL,C,D,STD,367204104,...,1.0,49,0,0,0,0,0,0,0,0
2,2,12994,10001,19920630,1992,INDL,C,D,STD,367204104,...,1.0,49,0,0,0,0,0,1,0,0
3,3,12994,10001,19930630,1993,INDL,C,D,STD,367204104,...,1.0,49,0,0,0,0,0,1,0,0
4,4,12994,10001,19940630,1994,INDL,C,D,STD,367204104,...,1.0,49,0,0,0,1,0,1,0,0
5,5,12994,10001,19950630,1995,INDL,C,D,STD,367204104,...,1.0,49,0,0,0,1,0,1,0,0
6,6,12994,10001,19960630,1996,INDL,C,D,STD,367204104,...,1.0,49,0,0,0,1,1,1,0,0
7,7,12994,10001,19970630,1997,INDL,C,D,STD,367204104,...,1.0,49,0,0,0,1,1,1,0,0
8,8,12994,10001,19980630,1998,INDL,C,D,STD,367204104,...,1.0,49,0,0,0,1,1,1,0,0
9,9,12994,10001,19990630,1999,INDL,C,D,STD,367204104,...,1.0,49,0,0,0,1,1,1,1,1
